In [6]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import random
import math
import random


logging.basicConfig(level=logging.DEBUG)

# List of city files to analyze
city_files = ['cities/vanuatu.csv', 'cities/italy.csv', 'cities/us.csv', 
              'cities/russia.csv', 'cities/china.csv' ] 

#Load city data from a CSV file and return it as a DataFrame.
def load_cities(file_path):
    return pd.read_csv(file_path, header=None, names=['name', 'lat', 'lon'])

#Create a distance matrix for the given city DataFrame.
def create_distance_matrix(cities):
    n = len(cities)
    dist_matrix = np.zeros((n, n))
    for c1, c2 in combinations(cities.itertuples(), 2):
        dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km
    return dist_matrix

In [7]:

#Solve TSP using a greedy nearest-neighbor approach and return the path and total cost.
def tsp_greedy(dist_matrix, cities):
    n = dist_matrix.shape[0]
    path = [0]  # Start from the first city (index 0)
    visited = set(path)
    total_cost = 0.0
    current_city = 0

    for _ in range(n - 1):
        next_city = None
        min_distance = float('inf')
        for j in range(n):
            if j not in visited and dist_matrix[current_city, j] < min_distance:
                min_distance = dist_matrix[current_city, j]
                next_city = j

        path.append(next_city)
        visited.add(next_city)
        total_cost += min_distance

        # Log each step
        '''logging.debug(
            f"Step: {cities.at[current_city, 'name']} -> {cities.at[next_city, 'name']} ({dist_matrix[current_city, next_city]:.2f} km)"
        )'''

        current_city = next_city

    # Return to the starting city and update the cost
    total_cost += dist_matrix[current_city, path[0]]
    path.append(path[0])  # Close the loop by returning to the starting city

    # Log final step
    '''logging.debug(
        f"Step: {cities.at[current_city, 'name']} -> {cities.at[path[0], 'name']} ({dist_matrix[current_city, path[0]]:.2f} km)"
    )'''

    return path, total_cost

In [8]:

# Main execution
for file_path in sorted(city_files):
    # Load city data and create distance matrix
    cities = load_cities(file_path)
    dist_matrix = create_distance_matrix(cities)

    # Solve TSP and get the path and total cost
    path, total_cost = tsp_greedy(dist_matrix, cities)

    # Print sorted results for each file
    print(f"\n=== Results for {file_path.split('/')[-1]} ===")
    '''for i in range(len(path) - 1):
        print(f"{cities.at[path[i], 'name']} -> {cities.at[path[i+1], 'name']} ({dist_matrix[path[i], path[i+1]]:.2f} km)")'''
    print(f"Total Path Cost: {total_cost:.2f} km\n")



=== Results for china.csv ===
Total Path Cost: 63962.92 km


=== Results for italy.csv ===
Total Path Cost: 4436.03 km


=== Results for russia.csv ===
Total Path Cost: 42334.16 km


=== Results for us.csv ===
Total Path Cost: 48050.03 km


=== Results for vanuatu.csv ===
Total Path Cost: 1475.53 km



In [9]:
#Simulated Annealing algorithm to solve the TSP
def simulated_annealing(dist_matrix, cities, iterations, initial_temp, final_temp, cooling_factor):
    
    n = len(dist_matrix)
    path = list(range(n))
    random.shuffle(path)

    def calculate_total_distance(path):
        return sum(dist_matrix[path[i], path[i + 1]] for i in range(n - 1)) + dist_matrix[path[-1], path[0]]

    current_distance = calculate_total_distance(path)
    best_distance = current_distance
    best_path = path[:]
    
    temperature = initial_temp
    
    while temperature > final_temp:
        for _ in range(iterations):
            new_path = path[:]
            i, j = sorted(random.sample(range(n), 2))
            new_path[i:j+1] = reversed(new_path[i:j+1])
            new_distance = calculate_total_distance(new_path)
            distance_delta = new_distance - current_distance

            # Condition to accept the new path
            if distance_delta < 0 or random.random() < math.exp(-distance_delta / temperature):
                path, current_distance = new_path, new_distance
                if current_distance < best_distance:
                    best_path, best_distance = path[:], current_distance
                    #logging.debug(f"New optimized path to {best_distance:.2f} km")

        # Cooling step
        temperature *= cooling_factor

    # Final log for the best path found
    # logging.debug("Best path found:")
    # for k in range(n - 1):
    #     logging.debug(
    #         f"{cities.at[best_path[k], 'name']} -> {cities.at[best_path[k + 1], 'name']} "
    #         f"({dist_matrix[best_path[k], best_path[k + 1]]:.2f} km)"
    #     )
    # logging.debug(
    #     f"{cities.at[best_path[-1], 'name']} -> {cities.at[best_path[0], 'name']} "
    #     f"({dist_matrix[best_path[-1], best_path[0]]:.2f} km)"
    # )
    
    return best_path, best_distance


In [10]:
for file in city_files:
    cities = load_cities(file)
    dist_matrix = create_distance_matrix(cities)

    # Parameters
    iterations = 500  # Originally 10000
    initial_temp = 100
    final_temp = 0.01
    cooling_factor = 0.99

    # Simulated Annealing
    logging.info("Running Simulated Annealing algorithm...")
    sa_path, sa_distance = simulated_annealing(dist_matrix, cities, iterations, initial_temp, final_temp, cooling_factor)
    print(f"Best path with Simulated Annealing: Distance: {sa_distance:.2f} km") 


INFO:root:Running Simulated Annealing algorithm...
INFO:root:Running Simulated Annealing algorithm...


Best path with Simulated Annealing: Distance: 1345.54 km
Best path with Simulated Annealing: Distance: 4172.76 km


INFO:root:Running Simulated Annealing algorithm...


Best path with Simulated Annealing: Distance: 40562.43 km


INFO:root:Running Simulated Annealing algorithm...


Best path with Simulated Annealing: Distance: 34763.86 km


INFO:root:Running Simulated Annealing algorithm...


Best path with Simulated Annealing: Distance: 62581.60 km
